In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [23]:
import numpy as np
import pandas as pd

import zipfile
import glob
import torch
import matplotlib.pylab as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv3D, Conv2DTranspose, MaxPooling2D, BatchNormalization, concatenate, Input, ConvLSTM2D,LSTM, Reshape
from tensorflow.keras import Model

import warnings
warnings.filterwarnings("ignore")

In [24]:
#GPU 사용 가능 여부 확인
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [25]:
tf.test.is_gpu_available()

In [26]:
#data load
train_files = glob.glob('../input/radar-image/train/train/*.npy')
len(train_files)

In [27]:
#data check
data_1st = np.load(train_files[0])
data_1st.shape

In [28]:
#train data visualization

#color map setting
color_map = plt.cm.get_cmap('RdBu')
color_map = color_map.reversed()

#visualization
plt.style.use('fivethirtyeight')
plt.figure(figsize = (20,20))

for i in range(4):
    plt.subplot(1,5,i+1)
    plt.imshow(data_1st[:,:,i],cmap = color_map)
    
plt.subplot(1,5,5)
plt.imshow(data_1st[:,:,-1], cmap=color_map)
plt.show()

In [29]:
#data preprocessing

def trainGenerator():
    for file in train_files:
        dataset = np.load(file)
        target = dataset[:,:,-1].reshape(120,120,1)
        remove_minus = np.where(target<0,0,target)
        feature = dataset[:,:,:4]
        
        yield (feature, remove_minus)

In [30]:
train_dataset = tf.data.Dataset.from_generator(trainGenerator, (tf.float32, tf.float32), (tf.TensorShape([120,120,4]),tf.TensorShape([120,120,1])))
train_dataset = train_dataset.batch(4).prefetch(1)

In [31]:
from keras.models import Model
from keras.layers import Input
from keras.layers.convolutional import Conv2D, ZeroPadding2D, Conv2DTranspose, UpSampling2D
from keras.layers.merge import concatenate, add, average
from keras.layers import LeakyReLU, BatchNormalization, Activation, Dropout, MaxPooling2D
from keras import regularizers


In [32]:
#unet model
def deepUnet_model(input_layer, start_neurons):
    
    #unet 1layer
    conv1 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(input_layer)
    bn1 = BatchNormalization()(conv1)
    conv1_2 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding = 'same')(bn1)
    bn1_2 = BatchNormalization()(conv1_2)  #(120,120,32)
    
    #shortcut part  #(120,120,32)
    shortcut1 = Conv2D(start_neurons *1 ,(1,1), activation='relu',padding='same')(input_layer)
    
    add1 = add([bn1_2, shortcut1]) #(120,120,64)
    add1 = Activation(activation = 'relu')(add1)
    pool1 = MaxPooling2D((2, 2))(add1)  #(60,60,64)

    #encoder 2layer  (60,60,64)
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    bn2 = BatchNormalization()(conv2)
    conv2_2 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding = 'same')(bn2)
    bn2_2 = BatchNormalization()(conv2_2)  #(60,60,32)
    
    #shortcut part  (60,60,32)
    shortcut2 = Conv2D(start_neurons *1 ,(1,1), activation='relu',padding='same')(pool1)
    
    add2 = add([bn2_2, shortcut2])  #(60,60,64)
    add2 = Activation(activation = 'relu')(add2)
    pool2 = MaxPooling2D((2, 2))(add2)  #(30,30,64)

    convm = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(add2)
    #(30,30,64)
    
    #(60,60,64)
    uconv2 = concatenate([convm, conv2])
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(deconv2)
    uconv2 = BatchNormalization()(uconv2)

    uconv1 = concatenate([uconv2, conv1])
    deconv1 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    
    output_layer = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1)
   
    return output_layer

    
input_layer = Input((120, 120, 4))
output_layer = deepUnet_model(input_layer,32)

In [33]:
with tf.device("/device:GPU:0"):
    model = Model(input_layer, output_layer)
    model.compile(loss='mae', optimizer='adam')

In [34]:
model.summary()

In [35]:
#gpu 사용하여 학습해보자
with tf.device("/device:GPU:0"):
    model.fit(train_dataset, epochs = 1, verbose=1)

In [36]:
from tensorflow.keras import backend as K

In [37]:
#unet model
def unet_model(input_layer, start_neurons):
    
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    pool1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D((2, 2))(pool1)

    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    pool2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D((2, 2))(pool2)

    convm = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)

    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = BatchNormalization()(uconv2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    
    output_layer = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1)
   
    return output_layer

    
in_layer = Input((120, 120, 4))
out_layer = unet_model(in_layer,64)

In [38]:
with tf.device("/device:GPU:0"):
    model2 = Model(in_layer, out_layer)
    model2.compile(loss='mae', optimizer='adam')

In [39]:
model2.summary()

In [40]:
#gpu 사용하여 학습해보자
with tf.device("/device:GPU:0"):
    model2.fit(train_dataset, epochs = 1, verbose=1)

In [41]:
class DeepUNet():
    def __init__(self):
        print("build deepUNet ...")
        
    def down_block(self, data, n_filter, name):
        x = MaxPooling2D(pool_size=(2,2), strides=(2,2))(data)
        temp = Conv2D(filters=n_filter, kernel_size = (3,3), strides = (1,1), padding = 'same', name = 'conv_{}'.format(name))(x)
        temp = Activation('relu')(temp)
        bn = BatchNormalization()(temp)
        
        bn = self.shortcut(x, bn)
        act = Activation('relu')(bn)
        return bn, act
    
    def up_block(self, act, bn, n_filter, name):
        x = UpSampling2D(size = (2,2), name = 'upsample_{}'.format(name))(act)
        
        temp = concatenate([bn, x], axis = 1)
        temp = Conv2D(filters=n_filter, kernel_size = (3,3), strides = (1,1), padding = 'same', name = 'conv_{}'.format(name))(x)
        temp = Activation('relu')(temp)
        
        bn = BatchNormalization()(temp)
        bn = self.shortcut(x, bn)
        act = Activation('relu')(bn)
        
        return act
        
    
    def shortcut(self, input, residual):
        
        input_shape = K.int_shape(input)
        
        try:
            residual_shape = np.shape(residual).as_list()
        except:
            residual_shape = np.shape(residual)
        
        
        
        #equal_width = input_shape[ROW_AXIS] == residual_shape[ROW_AXIS]
        #equal_heights = input_shape[COL_AXIS] == residual_shape[COL_AXIS]
        
        stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))
        stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))
        equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]
        
        
        shortcut = input
        # 1 X 1 conv if shape is different. Else identity.
        if stride_width > 1 or stride_height > 1 or not equal_channels:
            
            shortcut = Conv2D(filters = residual_shape[CHANNEL_AXIS],
                             kernel_size = (1,1),
                             strides = (stride_width, stride_height),
                             padding = 'valid',
                             kernel_initializer = 'he_normal',
                             kernel_regularizer = regularizers.l2(0.0001))(input)
        
        return add([shortcut, residual])
    
    
    
    def handle_dim_ordering(self):
        global ROW_AXIS
        global COL_AXIS
        global CHANNEL_AXIS
        
        ROW_AXIS = 1
        COL_AXIS = 2
        CHANNEL_AXIS = 3
        
    
    
    
    def create_model(self, img_shape, n_filter):
        
        self.handle_dim_ordering()
        K.set_learning_phase(True)
        
        inputs = Input(shape = img_shape)
        
        x = Conv2D(filters=n_filter, kernel_size = (3,3), strides = (1,1), padding = 'same', name = 'conv_{}'.format('conv0_1'))(inputs)
        net = Conv2D(filters=n_filter, kernel_size = (3,3), strides = (1,1), padding = 'same', name = 'conv_{}'.format('conv0_2'))(x)
        bn1 = BatchNormalization()(net)
        act1 = Activation('relu')(bn1)  #(120,120,64)
        
        bn2, act2 = self.down_block(act1, 64, 'down2') #(60,60,32)
        bn3, act3 = self.down_block(act2, 64, 'down3') #(30,30,32)
        bn4, act4 = self.down_block(act3, 64, 'down4') #(15,15,32)
        
        temp = self.up_block(act4, bn3, 64, 'up4') #(30,30,32)
        temp = self.up_block(temp, bn2, 64, 'up3') #(60,60,32)
        temp = self.up_block(temp, bn1, 64, 'up2') #(120,120,64)
        
        output = Conv2D(1, (1,1), padding="same", activation='relu')(temp) #(120,120,1)
        
        model = Model(outputs = output, inputs = inputs)
        print(model.summary())
        
        return model

In [42]:
with tf.device("/device:GPU:0"):
    model3 = DeepUNet().create_model((120, 120, 4),64)
    model3.compile(loss='mae', optimizer='adam')

In [43]:
#gpu 사용하여 학습해보자
with tf.device("/device:GPU:0"):
    model3.fit(train_dataset, epochs = 1, verbose=1)

In [44]:
#test 데이터 만들기
import random
#test_files = random.sample(train_files, int(len(train_files)*0.3))
test_files = random.sample(train_files, 2674)

In [46]:
len(test_files)

In [47]:
x_test = []
for file in tqdm(test_files, desc = 'test'):
    data = np.load(file)
    feature = data[:,:,:4]
    x_test.append(feature)
    
x_test = np.array(x_test)

In [48]:
y_test = []
for file in tqdm(test_files, desc = 'test'):
    data = np.load(file)
    target = data[:,:,-1].reshape(120,120,1)
    remove_minus = np.where(target<0,0,target)
    y_test.append(remove_minus)
    
y_test = np.array(y_test)

In [49]:
#모델 성능 확인하기
pred1 = model.predict(x_test)

In [51]:
#모델 성능 확인하기
pred2 = model2.predict(x_test)

In [52]:
#모델 성능 확인하기
pred3 = model3.predict(x_test)

In [53]:
#test data mae 측정
def MAE(y, pred):
    return np.mean(np.abs(y-pred))

In [54]:
print(MAE(y_test, pred1))

In [55]:
print(MAE(y_test, pred2))

In [56]:
print(MAE(y_test, pred3))

In [59]:
#시각화
#predict visualization

#color map setting
color_map = plt.cm.get_cmap('RdBu')
color_map = color_map.reversed()

#visualization
plt.style.use('fivethirtyeight')
plt.figure(figsize = (20,20))

plt.subplot(1,5,1)
plt.imshow(y_test[2], cmap = color_map)
plt.subplot(1,5,2)
plt.imshow(pred2[2], cmap=color_map)
plt.subplot(1,5,3)
plt.imshow(pred1[2], cmap = color_map)
plt.subplot(1,5,4)
plt.imshow(pred3[2], cmap=color_map)

plt.show()